In [1]:
spark.sparkContext.getConf().get('spark.driver.memory')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1607009916543_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'1000M'

In [2]:
%%configure -f 
{"driverMemory": "8000M"}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1607009916543_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1607009916543_0004,pyspark,idle,Link,Link,✔


In [3]:
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType, StringType, StructField, StructType
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-1.amazonaws.com")
sqlContext = SQLContext(sc)

match = sqlContext.read.format('com.databricks.spark.csv').option('delimiter', '\t').load('s3://millionsongprocessed/train_triplets.txt')
match = match.rdd.map(lambda p: (p[0], p[1], int(p[2])))

schemaTaste = StructType([StructField("user_id", StringType(), True), StructField("song_id", StringType(), True), StructField("playcount", IntegerType(), True)])
triplets = sqlContext.createDataFrame(match, schema=schemaTaste)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
triplets.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+
|             user_id|           song_id|playcount|
+--------------------+------------------+---------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|        1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|        2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|        1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|        1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|        1|
+--------------------+------------------+---------+
only showing top 10 rows

In [5]:
from pyspark.sql import functions as F
# read mismatches
mismatches = (
    spark.read.text('s3://millionsongprocessed/mismatch.txt')
    .select(
        F.trim(F.col('value').substr(9, 18)).alias('song_id').cast(StringType()),
        F.trim(F.col('value').substr(28, 18)).alias('track_id').cast(StringType())
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
mismatches.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+
|           song_id|          track_id|
+------------------+------------------+
|SOUMNSI12AB0182807|TRMMGKQ128F9325E10|
|SOCMRBE12AB018C546|TRMMREB12903CEB1B1|
|SOLPHZY12AC468ABA8|TRMMBOC12903CEB46E|
|SONGHTM12A8C1374EF|TRMMITP128F425D8D0|
|SONGXCA12A8C13E82E|TRMMAYZ128F429ECE6|
|SOMBCRC12A67ADA435|TRMMNVU128EF343EED|
|SOTDWDK12A8C13617B|TRMMNCZ128F426FF0E|
|SOEBURP12AB018C2FB|TRMMPBS12903CE90E1|
|SOSRJHS12A6D4FDAA3|TRMWMEL128F421DA68|
|SOIYAAQ12A6D4F954A|TRMWHRI128F147EA8E|
+------------------+------------------+
only showing top 10 rows

In [7]:
triplets = triplets.join(mismatches, on="song_id", how='left_anti')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
triplets.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+---------+
|           song_id|             user_id|playcount|
+------------------+--------------------+---------+
|SOAKIMP12A8C130995|b80344d063b5ccb32...|        1|
|SOAPDEY12A81C210A9|b80344d063b5ccb32...|        1|
|SOBBMDR12A8C13253B|b80344d063b5ccb32...|        2|
|SOBFNSP12AF72A0E22|b80344d063b5ccb32...|        1|
|SOBFOVM12A58A7D494|b80344d063b5ccb32...|        1|
|SOBNZDC12A6D4FC103|b80344d063b5ccb32...|        1|
|SOBSUJE12A6D4F8CF5|b80344d063b5ccb32...|        2|
|SOBVFZR12A6D4F8AE3|b80344d063b5ccb32...|        1|
|SOBXALG12A8C13C108|b80344d063b5ccb32...|        1|
|SOBXHDL12A81C204C0|b80344d063b5ccb32...|        1|
+------------------+--------------------+---------+
only showing top 10 rows

In [9]:
# count unique user_id
triplets.select('user_id').dropDuplicates().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1019318

In [10]:
# count unique song_id
triplets.select('song_id').dropDuplicates().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

378309

In [11]:
# most active users
active = (
    triplets
    .groupBy('user_id')
    .agg(
        F.sum('playcount').alias('playcount'),
        F.collect_list('song_id').alias('songs')
    )
    .orderBy('playcount', ascending=False)
    .limit(1)
    .rdd.take(1)[0]
)
active.__getattr__('user_id'), len(active.__getattr__('songs'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('093cb74eb3c517c5179ae24caf0ebec51b24d2a2', 195)

In [12]:
# song popularity
song_popularity = (
    triplets
    .groupBy('song_id')
    .agg(
        F.sum('playcount').alias('playcount')
    )
)

n = song_popularity.approxQuantile('playcount', [0.0, 0.25, 0.5, 0.75, 1.0], 0.0)
  # [1.0, 8.0, 31.0, 130.0, 726885.0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
n

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1.0, 8.0, 31.0, 130.0, 726885.0]

In [13]:
# user popularity
user_activity = (
    triplets
    .groupBy('user_id')
    .agg(
        F.count('song_id').alias('songcount')
    )
)

m = user_activity.approxQuantile('songcount', [0.0, 0.25, 0.5, 0.75, 1.0], 0.0)
  # [3.0, 15.0, 26.0, 53.0, 4316.0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
m

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[3.0, 15.0, 26.0, 53.0, 4316.0]

In [14]:
# inactive songs
inactive_song = (
    song_popularity
    .filter(song_popularity.playcount < n[1])
    .select('song_id')
)

# inactive users
inactive_user = (
    user_activity
    .filter(user_activity.songcount < m[1])
    .select('user_id')
)

# remove inactives from triplets
triplets = (
    triplets
    .join(inactive_song, on='song_id', how='left_anti')
    .join(inactive_user, on='user_id', how='left_anti')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
inactive_song.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

90750

In [15]:
# count
triplets.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

42818160

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, QuantileDiscretizer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RankingMetrics

# ALS Recomm
# convert user and song into an integer index
indexer_user = StringIndexer(inputCol='user_id', outputCol='user')
indexer_song = StringIndexer(inputCol='song_id', outputCol='item')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# pipeline
pipeline = Pipeline(stages=[indexer_user, indexer_song]) 
pipelineModel = pipeline.fit(triplets)
dataset = pipelineModel.transform(triplets)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# convert playcount into double (as rating)
dataset = (
    dataset
    .withColumn('user', F.col('user').cast(IntegerType()))
    .withColumn('item', F.col('item').cast(IntegerType()))
    .withColumn('rating', F.col('playcount').cast(IntegerType()))
    .select(['user', 'item', 'rating'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# create dictionary of fraction 30% for each user
f = (
    dataset
    .select('user')
    .dropDuplicates()
    .withColumn('temp', F.lit(0))
    .groupBy('temp')
    .agg(F.collect_list('user').alias('list'))
    .select('list')
    .rdd.take(1)[0].__getattr__('list')
)

fractions = dict(
    (user, 0.3) for user in f
)

# sample test set using fractions
test = dataset.sampleBy('user', fractions, seed=1)

# get training set by remove test set from full dataset
training = (
    dataset
    .join(
        test
        .select(['user', 'item']),
        on=['user', 'item'],
        how='left_anti'
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# select three users
test.orderBy('rating', ascending=False).show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+------+
|user  |item  |rating|
+------+------+------+
|519321|30302 |2381  |
|208949|48057 |2368  |
|427453|62    |1729  |
|48502 |1133  |1646  |
|95081 |5178  |1620  |
|489344|112166|1539  |
|148063|11467 |1467  |
|60684 |45731 |1436  |
|117146|6017  |1222  |
|176060|5462  |1160  |
+------+------+------+
only showing top 10 rows

In [24]:
# create als model / Set the seed. 
als = ALS(seed=1)

# fit training set and tranform test set
model = als.fit(training)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
from pyspark.sql.types import ArrayType
u = [519321, 208949, 427453]
users = sqlContext.createDataFrame(
    [(519321,), (208949,), (427453,)],
    StructType([StructField('user', IntegerType())])
)

# recommend for users
recommends = model.recommendForUserSubset(users, 10)

# get prediction and labels
def recommend(recommendations):
    items = []
    for item, rating in recommendations:
        items.append(item)
    return items

udf_recommend = F.udf(lambda recommendations: recommend(recommendations), ArrayType(IntegerType()))

recommends = (
    recommends
    .withColumn('recommends', udf_recommend(recommends.recommendations))
    .select(
        F.col('user').cast(IntegerType()),
        F.col('recommends')
    )
)

recommends.show(3, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------------------------------------------------------------------+
|user  |recommends                                                                    |
+------+------------------------------------------------------------------------------+
|208949|[112166, 48057, 160211, 108002, 207956, 67531, 220834, 195067, 276721, 257720]|
|519321|[160211, 68319, 30302, 194353, 67531, 129848, 220834, 20037, 29730, 179474]   |
|427453|[112166, 166446, 217213, 251809, 61040, 47969, 67531, 71739, 269306, 237084]  |
+------+------------------------------------------------------------------------------+

In [26]:
# get labels
ground_truths = (
    test
    .filter(F.col('user').isin(u))
    .orderBy('rating', ascending=False)
    .groupBy('user')
    .agg(F.collect_list('item').alias('ground_truths'))
)

ground_truths.show(3, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------------------------------------------------------------------------------------------------------------------------------+
|user  |ground_truths                                                                                                                           |
+------+----------------------------------------------------------------------------------------------------------------------------------------+
|208949|[21565, 120303, 4589, 10868, 1780, 0, 2642, 72724, 1276, 171, 21961, 2925, 41137, 36614, 1073, 32810, 40332, 104026, 6101, 103178, 5256]|
|519321|[21789, 33405, 85783, 73478, 117819, 121165, 86341, 79604, 10952, 45869, 22227, 17408]                                                  |
|427453|[414, 4, 23067, 29, 185, 28977, 431, 82, 81, 62]                                                                                        |
+------+--------------------------------------------------------------------------------------------------------------------

In [27]:
compare = recommends.join(ground_truths, on='user', how='left')
compare = [(r.__getattr__('recommends'), r.__getattr__('ground_truths')) for r in compare.collect()]
compare = sc.parallelize(compare)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# print metrics
metrics = RankingMetrics(compare)
print(metrics.precisionAt(10))

print(metrics.ndcgAt(10))

print(metrics.meanAveragePrecision)


# predict test and rmse
predict = model.transform(test)
predict = predict.filter(F.col('prediction') != float('nan'))
reg_eval = RegressionEvaluator(predictionCol='prediction', labelCol='rating', metricName='rmse')
reg_eval.evaluate(predict)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.0
0.0
0.0
4.856587012747